<center><img width="70%" src="https://www.vivacidadania.org.br/wp-content/uploads/2019/02/observat%C3%B3rio-social-1200x675.jpg"></center>

# Capturando os dados do site

Importando todas as bibliotecas que serão utilizadas

In [ ]:
import time
import pandas as pd
import bs4
import pyodbc
import warnings

from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen

---

In [ ]:
agente = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.63'
headers = {'User-Agent': agente}

---

Criando uma função pra acessar a página da prefeitura

In [ ]:
def Acesso(url):
    
    try:
        req = Request(url, headers = headers) # Metodo Request pra fazer uma solicitação
        response = urlopen(req)
        return response.read()
        
    except:
        pass

---

Criando uma função para capturar a página que acessamos e tranforma-las em padrão html

In [ ]:
def captura_html_pagina(url):
    
    html = Acesso(url) # Usando a função pra pegarmos o html na função Acesso
    soup = bs4.BeautifulSoup(html, 'html.parser') # Modelamos o html em modelo html, pois o response deixa tudo bagunçado
    return soup # retornamos a variavel modelo de HTML

---

Identifiquei que no código fonte do Site, a parte que queremos pegar só existe ela, ou seja, a proposição, o ano, o texto, o autor, estão dentro de uma classe única, o que facilitou o meu trabalho.

Então vamos pegar o cabecalho da pagina, ou seja, vamos buscar apenas as informações que queremos tirar do site

In [ ]:
def Cabecalho(html):
    dt = html.find_all('dt') # Procurando dentro do HTML a String 'dt' que é onde fica os Títulos do que buscamos
    dd = html.find_all('dd') # Procurando dentro do HTML a String 'dd' que é onde fica os valores dos títulos do que buscamos
    
    dic = {} # criando um dicionário que neles iremos inserir os dados buscados acima
    
    for i in range(len(dt)): # criando um for pra que ele comece no 0 e pare no máximo de Títulos que tivermos
        x = dt[i].get_text() # atribuindo a variavel x o elemento posição i só que apenas o texto
        y = dd[i].get_text() # atribuindo a variavel y o elemento posição i só que apenas o texto
        dic[x] = y # adicionando as variavéis que criamos acima no dicionario criado fora do for
    
    return dic # retornando o dicionário

---

Já capturamos a pagina selecionamos o cabeçalho com as informações que queremos, vamos agora captar o conteudo

In [ ]:
def Conteudo(proposicao, ano):
    # criamos uma variavel chamada url que contém o site e na parte onde diz sobre a proposicao e ano, atribuimos as variaveis
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url) # chamamos a função para pegar o url acima e já captar o site no formato HTML
    dic = Cabecalho(html) # Retirarmos as informações de cabeçalho que queremos
    dic['Proposição'] = proposicao # atribuimos as informações acima os números da proposição e ano que atribuimos na função
    dic['Ano'] = ano 
    dic['Texto'] = html.p.get_text()
    return dic # retornamos o dicionário

---

Irei fazer o teste com a 1º proposição do ano de 2023

In [ ]:
# criei uma lista vazia que será adicionada a proposição de acordo com a função Conteudo
lista = [] 

lista.append(Conteudo(1, 2023))

# atribuir a lista a um Data Frame chamado teste e mostrei na tela o resultdo
teste = pd.DataFrame(lista) 
teste

Percebemos que conseguimos atribuir a 1º proposição do ano de 2023 ao DataFrame e já visualizamos de uma forma mais fácil de se entender.

Então agora vamos...

---

# Conectar os dados ao SQL

A ponto de facilitar mais o código irei criar funções separadas também

A primeira função vai ser de (Select) que é apenas para fazer uma consulta no banco de dados dentro do SQL através do Python


In [ ]:
def SQLSelect(query):
    
    conn = pyodbc.connect('Trusted_Connection=yes',
                         driver = '{ODBC Driver 17 for SQL Server}',
                         server = 'localhost',
                         database = 'Indaia')
    
    out = pd.read_sql_query(query, conn)
    
    return out

---

A segunda função vai ser de Truncate, ou seja apagar todos os dados existentes dentro da tabela, caso precisamaremos


In [ ]:
def Truncate(NomeDaTabela):
    
    conn = pyodbc.connect('Trusted_Connection=yes',
                         driver = '{ODBC Driver 17 for SQL Server}',
                         server = 'localhost',
                         database = 'Indaia')
    
    cursor = conn.cursor()
    
    cursor.execute(f'''
    
            TRUNCATE TABLE {NomeDaTabela}
            
            ''')
    
    conn.commit()
    cursor.close()

---

A terceira função desta etapa é a de inserir elementos no banco de dados


In [ ]:
def SQLInsert(TabelaProposicoes):
    
    base = pd.DataFrame(columns = ['Reuniao','Deliberacao','Situacao','Assunto','Autor','Proposicao','Ano','Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')
    
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaia')
    
    cursor = conn.cursor()
    
    for index, row in TabelaProposicoes.iterrows():
        cursor.execute('''
             INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            ) 
            values(?,?,?,?,?,?,?,?)''', 

            row['Reunião'], 
            row['Deliberação'], 
            row['Situação'], 
            row['Assunto'], 
            row['Autor'], 
            row['Proposição'], 
            row['Ano'], 
            row['Texto']
            )
    cursor.commit()
    cursor.close() 

Essa função é pra inserirmos sempre a próxima proposição e caso caia no resultado em que o começo seja Nulo, a primeira proposição vai ser 1

In [ ]:
def InsereProximaProposicao(ano):

    # Busca última proposição cadastrada
    dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    # Verifica se foi identificado lançamento naquele ano
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1 

    # Captura e Insere dados na tabela
    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsert(tabela)

---

Nessa aqui estamos já fazendo um loop pra adicionar todas as proposições de um ano

In [ ]:
def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 2, segundos_espera = 0):

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while erros <= erros_admissiveis:

        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        i +=1

---

Agora iremos criar 2 loops do ano inicial até o ano final

In [ ]:
warnings.filterwarnings("ignore")
for i in range(1996, 2000):
    try:
        # Chamada da função
        BuscaGravaDadosAno(i)
    except Exception as e:
        print(f"Erro ao executar a função BuscaGravaDadosAno({i}): {e}")
    else:
        # Mensagem impressa somente após a conclusão da função
        print(f"Concluída a execução do ano({i})")
        # Impressão do próximo ano
        print(f"Próximo ano: {i + 1}")


In [ ]:
warnings.filterwarnings("ignore")
for i in range(2000, 2023):
    try:
        # Chamada da função
        BuscaGravaDadosAno(i)
    except Exception as e:
        print(f"Erro ao executar a função BuscaGravaDadosAno({i}): {e}")
    else:
        # Mensagem impressa somente após a conclusão da função
        print(f"Concluída a execução do ano({i})")
        # Impressão do próximo ano
        print(f"Próximo ano: {i + 1}")


Após todos esses passos iremos abrir no PowerBI e importar esses dados do SQLSERVER que inserimos e vamos fazer a nossa análise.